In [ ]:
# Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz.1

In [ ]:
# Настройка переменных окружения
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
# Установка findspark
!pip install -q findspark

In [ ]:
# Инициализация findspark
import findspark
findspark.init()

**Проверка установки. Инициализация сессии, импорт необходимых далее библиотек**

In [ ]:
# Создание SparkContext (проверка установки)
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

sc = spark.sparkContext

print(sc.version)

3.1.1


**Загрузка данных**

In [ ]:
# Создаем SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# Читаем файлы CSV
trips_df = spark.read.csv("trips.csv", header=True, inferSchema=True)  # header=True, чтобы использовать заголовки столбцов
stations_df = spark.read.csv("stations.csv", header=True, inferSchema=True)

# Отображаем по 5 строк из каждого файла
trips_df.show(n=5)
stations_df.show(n=5)

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|           null|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|    null|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...|              27|8/29/2013 10:17|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|8/29/2013 11:29|  San Jose City Hall|      

In [ ]:
# выводит схему данных датафрейма spark соответственно (имена столбцов, типы данных и информация о том, допускает ли столбец значения NULL)
print("Trips")
trips_df.printSchema()  # Используем trips_df

print("Stations")
stations_df.printSchema() # Используем stations_df

Trips
root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

Stations
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



# Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
1. Найти велосипед с максимальным временем пробега.
2. Найти наибольшее геодезическое расстояние между станциями.
3. Найти путь велосипеда с максимальным временем пробега через станции.
4. Найти количество велосипедов в системе.
5. Найти пользователей потративших на поездки более 3 часов.

**1. Найти велосипед с максимальным временем пробега**

In [ ]:
# Найдите максимальное время пробега (duration)
max_duration = trips_df.agg(max("duration")).collect()[0][0]

# Найдите велосипед (bike_id) с этим максимальным временем пробега
bike_with_max_duration_df = trips_df.filter(trips_df.duration == max_duration)

# Выведите результаты
print("Велосипед с максимальным временем пробега:")
bike_with_max_duration_df.show()  # Показать всю информацию о поездке/поездках

# Получить ID велосипеда из DataFrame
bike_id = bike_with_max_duration_df.select("bike_id").first()[0]

print(f"ID велосипеда: {bike_id}")
print(f"Максимальное время пробега: {max_duration}")


Велосипед с максимальным временем пробега:
+------+--------+---------------+--------------------+----------------+---------------+----------------+--------------+-------+-----------------+--------+
|    id|duration|     start_date|  start_station_name|start_station_id|       end_date|end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+------+--------+---------------+--------------------+----------------+---------------+----------------+--------------+-------+-----------------+--------+
|568474|17270400|12/6/2014 21:59|South Van Ness at...|              66|6/24/2015 20:18|   2nd at Folsom|            62|    535|         Customer|   95531|
+------+--------+---------------+--------------------+----------------+---------------+----------------+--------------+-------+-----------------+--------+

ID велосипеда: 535
Максимальное время пробега: 17270400


**2. Найти наибольшее геодезическое расстояние между станциями**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, radians, sin, cos, sqrt, atan2, col

# Создаем SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# Читаем файлы CSV
stations_df = spark.read.csv("stations.csv", header=True, inferSchema=True)

# Создаем пары станций
station_pairs = stations_df.alias("station1").crossJoin(stations_df.alias("station2")).filter(col("station1.id") < col("station2.id"))

# Вычисляем максимальное расстояние между станциями
max_distance = station_pairs.select(
    (
        2 * 6371.0 * atan2(
            sqrt(
                sin(radians(col("station2.lat") - col("station1.lat")) / 2)**2 +
                cos(radians(col("station1.lat"))) * cos(radians(col("station2.lat"))) *
                sin(radians(col("station2.long") - col("station1.long")) / 2)**2
            ),
            sqrt(1 - (
                sin(radians(col("station2.lat") - col("station1.lat")) / 2)**2 +
                cos(radians(col("station1.lat"))) * cos(radians(col("station2.lat"))) *
                sin(radians(col("station2.long") - col("station1.long")) / 2)**2
            ))
        )
    ).alias("distance")
).agg(max("distance")).collect()[0][0]

print(f"Максимальное геодезическое расстояние между станциями равно {max_distance:.2f} километрам")


Максимальное геодезическое расстояние между станциями равно 69.92 километрам


**3. Найти путь велосипеда с максимальным временем пробега через станции**

In [ ]:
# Сортировка по столбцу duration и выбор наиболее длительной поездки
max_duration = trips_df.select("start_station_name", "end_station_name", "duration").orderBy(col("duration").desc()).first()

# Получение стартовой и конечной станций, времени поездки
start_location = max_duration["start_station_name"]
end_location = max_duration["end_station_name"]
trip_time = max_duration["duration"]

print(f"Самая длинная поездка: {trip_time} секунд.")
print(f"Велосипедист ехал из \"{start_location}\" в \"{end_location}\"")

Самая длинная поездка: 17270400 секунд.
Велосипедист ехал из "South Van Ness at Market" в "2nd at Folsom"


**4. Найти количество велосипедов в системе**

In [ ]:
# Находим количество уникальных велосипедов в системе
unique_bikes_count = trips_df.select("bike_id").distinct().count()
print(f"Суммарное количество велосипедов: {unique_bikes_count}")

Суммарное количество велосипедов: 700


**5. Найти пользователей потративших на поездки более 3 часов**

In [ ]:
# Фильтруем пользователей, отбираем потративших более 3 часов (10800 секунд) на поездки
long_duration_trips = trips_df.filter(col("duration") > 10800)
# Отображаем результаты
long_duration_trips.show()

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4639|   11118|8/29/2013 15:18|       Market at 4th|              76|8/29/2013 18:23|       Market at 4th|            76|    433|         Customer|    null|
|4637|   11272|8/29/2013 15:17|       Market at 4th|              76|8/29/2013 18:25|       Market at 4th|            76|    377|         Customer|    null|
|4528|   12280|8/29/2013 13:39|Paseo de San Antonio|               7|8/29/2013 17:04|    Adobe on Almaden|             5|    645|       Subscriber|   94536|
|4363|   15244|8/29/2013 12:23|       Market at 4th|      

In [ ]:
# Остановка SparkSession
spark.stop()